In [1]:
!nvidia-smi

Wed Jun  8 09:05:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.94       Driver Version: 470.94       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   38C    P0   103W / 350W |      0MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:82:00.0 Off |                  N/A |
| 30%   

In [4]:
from tensorflow.keras.applications import ResNet50

BATCH_SIZE = 1024

In [7]:
model = ResNet50(weights='imagenet')

2022-06-02 22:43:04.588203: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 22:43:05.847185: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6
2022-06-02 22:43:05.848995: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22302 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:82:00.0, compute capability: 8.6


In [5]:
import numpy as np
from skimage import io
from skimage.transform import resize
# from matplotlib import pyplot as plt

url='n02099601_3004.jpg'
img = resize(io.imread(url), (224, 224))
input_batch = 255*np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

input_batch.shape

(32, 224, 224, 3)

In [8]:
predictions = model.predict(input_batch) # warm up
indices = (-predictions[0]).argsort()[:5]
print("Class | Probability (out of 1)")
list(zip(indices, predictions[0][indices]))

2022-06-02 22:43:13.998373: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-02 22:43:15.704411: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-06-02 22:43:16.886902: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Class | Probability (out of 1)


2022-06-02 22:43:18.268654: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[(160, 0.32256573),
 (169, 0.23734361),
 (212, 0.18466648),
 (170, 0.08061244),
 (207, 0.030543853)]

In [9]:
%%timeit

result = model.predict_on_batch(input_batch) # Check default performance

65.1 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
import onnx

In [13]:
model.save('my_model')
!python -m tf2onnx.convert --saved-model my_model --output temp.onnx
onnx_model = onnx.load_model('temp.onnx')

INFO:tensorflow:Assets written to: my_model/assets


/root/miniconda3/envs/tf27/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


/root/miniconda3/envs/tf27/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-06-02 22:51:28,609 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-06-02 22:51:35,434 - INFO - Signatures found in model: [serving_default].
2022-06-02 22:51:35,434 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-06-02 22:51:35,436 - INFO - Output names: ['predictions']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-06-02 22:51:38,970 - WARNING - From /root/miniconda3/envs/tf27/lib/python3.9/site-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [14]:
inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = BATCH_SIZE

In [15]:
model_name = "resnet50_onnx_model_tf.onnx"
onnx.save_model(onnx_model, model_name)
print("Done saving!")

Done saving!


In [16]:
import os, time
print("Restarting kernel  in three seconds...")
time.sleep(3)
print("Restarting kernel now")
os._exit(0) # Shut down all kernels so TRT doesn't fight with Tensorflow fo

Restarting kernel  in three seconds...


: 

: 

In [1]:
BATCH_SIZE = 32

In [2]:
import numpy as np

USE_FP16 = True

target_dtype = np.float16 if USE_FP16 else np.float32

In [3]:
import numpy as np
from skimage import io
from skimage.transform import resize
# from matplotlib import pyplot as plt

url='n02099601_3004.jpg'
img = resize(io.imread(url), (224, 224))
input_batch = 255*np.array(np.repeat(np.expand_dims(np.array(img, dtype=np.float32), axis=0), BATCH_SIZE, axis=0), dtype=np.float32)

input_batch.shape

(32, 224, 224, 3)

In [4]:
input_batch = input_batch.astype(target_dtype)

In [7]:
# May need to shut down all kernels and restart before this - otherwise you might get cuDNN initialization errors:
if USE_FP16:
    !trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt  --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16
else:
    !trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt  --explicitBatch

&&&& RUNNING TensorRT.trtexec [TensorRT v8204] # trtexec --onnx=resnet50_onnx_model_tf.onnx --saveEngine=resnet_engine.trt --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16
[06/02/2022-22:59:06] [W] --explicitBatch flag has been deprecated and has no effect!
[06/02/2022-22:59:06] [W] Explicit batch dim is automatically enabled if input model is ONNX or if dynamic shapes are provided when the engine is built.
[06/02/2022-22:59:06] [I] === Model Options ===
[06/02/2022-22:59:06] [I] Format: ONNX
[06/02/2022-22:59:06] [I] Model: resnet50_onnx_model_tf.onnx
[06/02/2022-22:59:06] [I] Output:
[06/02/2022-22:59:06] [I] === Build Options ===
[06/02/2022-22:59:06] [I] Max batch: explicit batch
[06/02/2022-22:59:06] [I] Workspace: 16 MiB
[06/02/2022-22:59:06] [I] minTiming: 1
[06/02/2022-22:59:06] [I] avgTiming: 8
[06/02/2022-22:59:06] [I] Precision: FP32+FP16
[06/02/2022-22:59:06] [I] Calibration: 
[06/02/2022-22:59:06] [I] Refit: Disabled
[06/02/2022-22:59:06] [I] Spa

In [10]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

f = open("resnet_engine.trt", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

[06/02/2022-23:02:45] [TRT] [W] TensorRT was linked against cuBLAS/cuBLASLt 11.6.5 but loaded cuBLAS/cuBLASLt 11.5.2
[06/02/2022-23:02:46] [TRT] [W] TensorRT was linked against cuBLAS/cuBLASLt 11.6.5 but loaded cuBLAS/cuBLASLt 11.5.2


In [11]:
output = np.empty([BATCH_SIZE, 1000], dtype = target_dtype) # Need to set output dtype to FP16 to enable FP16

# Allocate device memory
d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

In [12]:
def predict(batch): # result gets copied into output
    # Transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # Execute model
    context.execute_async_v2(bindings, stream.handle, None)
    # Transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # Syncronize threads
    stream.synchronize()
    
    return output

In [13]:
print("Warming up...")

trt_predictions = predict(input_batch).astype(np.float32)

print("Done warming up!")

Warming up...
Done warming up!


In [14]:
indices = (-trt_predictions[0]).argsort()[:5]
print("Class | Probability (out of 1)")
list(zip(indices, trt_predictions[0][indices]))

Class | Probability (out of 1)


[(160, 0.30859375),
 (169, 0.24609375),
 (212, 0.19018555),
 (170, 0.078063965),
 (207, 0.03152466)]

In [15]:
%%timeit

_ = predict(input_batch) # Check TRT performance

5.86 ms ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
